In [15]:
import os

import imageio
import pandas as pd
from keras_preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import os

import imageio
import pandas as pd
from keras import Input, Model
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.optimizers import Adam
import numpy as np
import keras.utils
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.model_selection import train_test_split
from keras.layers import (Conv2D, Flatten, Lambda, Dense, concatenate,
                          Dropout, Input)
from keras.models import Model
import numpy as np
from keras.models import Model
from keras.layers import Input, Conv2D, Activation, Flatten, Dense
from keras.optimizers import Adam


data_image = "C:\\Users\\Priit\\Dropbox\\Informaatika\\Magister\\Tehisnärvivõrgud\\data\\train_images"
data_dir = "C:\\Users\\Priit\\Dropbox\\Informaatika\\Magister\\Tehisnärvivõrgud\\data"

label_column = "AdoptionSpeed"

LIMIT = 30

test_size = 0.2


In [16]:
train = pd.read_csv(os.path.join(data_dir, "train.csv"), sep=',', nrows=LIMIT if LIMIT != -1 else None)
pet_ids = train["PetID"]

selected_columns = ["Type",
                    "Gender",
                    "Color1",
                    "Color2",
                    "Color3",
                    "MaturitySize",
                    "FurLength",
                    "Vaccinated",
                    "Dewormed",
                    "Sterilized",
                    "Health",
                    "State",
                    "MaturitySize"]




y = train[label_column]

# One-Hot-encode
# X = pd.get_dummies(train[selected_columns], columns=selected_columns)

# Normalize:
# to_normalize = ["Age", "Fee", "Quantity"]
# for to_norm in to_normalize:
#     X[to_norm] = (train[to_norm] - train[to_norm].mean()) / train[to_norm].std()

In [17]:
from PIL import Image
from skimage.transform import resize

height, width = 320, 320
image_paths = [os.path.join(data_dir, data_image, pet_id + "-1.jpg") for pet_id in pet_ids][:LIMIT]

images = []

for path in tqdm(image_paths):
    if os.path.isfile(path):
        image = Image.open(path).convert("RGB") 
        image.load()
        image = np.asarray(image, dtype="int32" )
        image = resize(image, (height, width), anti_aliasing=True, mode='constant')
    else:
        image = np.random.rand(height, width, 3) * 255
        
    images.append(image)
    
images = np.array(images)

# Standardize:
mean = np.mean(images)
std = np.std(images)

images_meanstd = (images - mean)/std

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 31.83it/s]


In [18]:
X_train, X_test, y_train, y_test = train_test_split(images_meanstd, y, test_size=test_size, random_state=1)

In [19]:
inputs = Input(shape=(height, width, 3))
h = Conv2D(32, (14, 14), strides=(2, 2))(inputs)
h = Conv2D(32, (7, 7), strides=(2, 2))(inputs)
h = Activation('relu')(h)
h = Flatten()(h)
h = Dense(100)(h)
h = Dense(100)(h)
h = Dense(5)(h)
predictions = Activation('softmax')(h)

model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


history = model.fit(X_train, 
                    keras.utils.to_categorical(y_train),
                    batch_size=64, 
                    epochs=5, 
                    validation_split=0.2)

Train on 19 samples, validate on 5 samples
Epoch 1/5
19/19 [==============================] - 7s 343ms/step - loss: 1.9462 - acc: 0.1053 - val_loss: 9.6709 - val_acc: 0.4000
Epoch 2/5
19/19 [==============================] - 2s 129ms/step - loss: 8.4832 - acc: 0.4737 - val_loss: 9.6709 - val_acc: 0.4000
Epoch 3/5
19/19 [==============================] - 3s 135ms/step - loss: 8.4832 - acc: 0.4737 - val_loss: 9.6709 - val_acc: 0.4000
Epoch 4/5
19/19 [==============================] - 2s 131ms/step - loss: 8.4832 - acc: 0.4737 - val_loss: 9.6709 - val_acc: 0.4000
Epoch 5/5
19/19 [==============================] - 3s 133ms/step - loss: 8.4832 - acc: 0.4737 - val_loss: 9.6709 - val_acc: 0.4000


In [20]:
test_predictions = model.predict(X_test)
test_predictions = [np.argmax(pred) for pred in test_predictions]
print(test_predictions)

train_pred = [np.argmax(pred) for pred in model.predict(X_train)]

print("Kappa on train: {}".format(round(cohen_kappa_score(y_train, train_pred, weights="quadratic"), 4)))
print("Accuracy on train: {}".format(round(accuracy_score(y_train, train_pred), 4)))
print("________________")
print("Kappa on test: {}".format(round(cohen_kappa_score(y_test, test_predictions, weights="quadratic"), 4)))
print("Accuracy on test: {}".format(round(accuracy_score(y_test, test_predictions), 4)))

[2, 2, 2, 2, 2, 2]
Kappa on train: 0.45
Accuracy on train: 0.4583
________________
Kappa on test: 0.0
Accuracy on test: 0.3333


In [ ]:
# https://stackoverflow.com/questions/49618986/neural-network-in-keras-with-two-different-input-types-images-and-values
# TODO: read images in
# TODO: read features in
image = np.random.rand(10, 66, 200, 3)
feature = np.random.rand(10, 3)  # feature vector
y = np.random.normal(0, 1, (10, 1))

image_input = Input(shape=(66, 200, 3))
aux_input = Input(shape=(3,))

lamb = Lambda(lambda x: x / 127.5 - 1.0, input_shape=(66, 200, 3))(image_input)
cov1 = Conv2D(24, 5, 5, activation='elu', subsample=(2, 2))(lamb)
cov2 = Conv2D(36, 5, 5, activation='elu', subsample=(2, 2))(cov1)
cov3 = Conv2D(48, 5, 5, activation='elu', subsample=(2, 2))(cov2)
cov4 = Conv2D(64, 3, 3, activation='elu')(cov3)
cov5 = Conv2D(64, 3, 3, activation='elu')(cov4)
dropout = Dropout(0.5)(cov5)
flatten = Flatten()(dropout)

# Here we add in the feature vectors
merge = concatenate([flatten, aux_input])

d1 = Dense(100, activation='elu')(merge)
d2 = Dense(50, activation='elu')(d1)
d3 = Dense(10, activation='elu')(d2)
out = Dense(1)(d3)

model = Model(inputs=[image_input, aux_input], outputs=[out])
model.compile(loss='mse', optimizer='rmsprop')
model.fit([image, feature], y)
